Exporting FastAI ResNet model to ONNX is a 2 step process - there's no native FastAI to ONNX export. Since, FastAI is a high-level API built on top of PyTorch - first we need to extract the PyTorch model. Only after that it's possible to get our ONNX model. Couple of things must be considered which I'll talk about and show later.

Steps to be taken:

1. Extract PyTorch model from FastAI
2. Convert PyTorch model to ONNX

In [1]:
from fastai.vision.all import *

In [2]:
# load FastAI learner
learn = load_learner('models/birds-res34-fast2.pkl')

## Test inference with FastAI learner

In [3]:
# roadrunner
print(learn.predict('test.jpg'))

('ROADRUNNER', tensor(422), tensor([6.5983e-14, 2.2859e-13, 5.9723e-12, 7.7789e-13, 1.5575e-12, 3.2188e-14,
        7.9489e-15, 1.3908e-10, 3.5392e-16, 8.0554e-15, 8.7294e-14, 4.9873e-14,
        1.5486e-14, 1.5034e-12, 9.4583e-14, 6.2127e-13, 6.7510e-15, 4.8466e-15,
        7.4699e-15, 5.3998e-14, 7.7535e-16, 1.0335e-11, 2.3246e-13, 1.1478e-13,
        4.0351e-14, 4.7952e-15, 1.2747e-14, 3.7469e-14, 4.8960e-13, 2.5345e-11,
        2.9240e-15, 1.8969e-16, 3.9972e-11, 3.6214e-14, 2.6427e-15, 2.2619e-13,
        6.9356e-12, 9.2867e-16, 8.5566e-13, 1.8381e-14, 5.1385e-15, 2.0151e-13,
        1.2745e-13, 5.1696e-11, 4.9146e-10, 4.1472e-11, 2.5322e-13, 1.6292e-14,
        1.3606e-11, 1.6115e-14, 9.0644e-17, 9.7111e-14, 1.5005e-13, 3.6555e-15,
        9.6599e-13, 3.9306e-13, 8.2510e-12, 2.9404e-15, 2.1990e-09, 1.0551e-13,
        3.6785e-14, 4.3111e-15, 9.5979e-11, 2.9967e-15, 7.0435e-12, 2.3685e-10,
        8.6018e-13, 4.7681e-13, 2.7982e-12, 8.5201e-12, 1.5925e-15, 1.1203e-12,
        5.54

## 1. Getting the PyTorch model

By using .model attribute on the FastAI learner we get the 'pure' PyTorch model. By using eval() we are setting the model to 'prediction' mode - backward propagation is disabled.

#### Note:
FastAI learner wraps the PyTorch model with additional operations defined in `DataBlock` in this case we defined a `Resize` operation. By default, FastAI learner also includes a softmax layer and a normalization layer.

### What does this mean ?
If we run `learn.predict('path/to/image')` our FastAI learner resizes the image to the size defined in our `DataBlock`, normalizes the color channels, passes the image through the neural net, scales the inference results between 0-1.

If we'd try to run inference on a PyTorch model extracted from a FastAI wrapper it would fail (most likely) as the image has an incorrect resolution. That's why we need to add transformations to a PyTorch models before inference.

In [4]:
# transformations performed on data loaders
learn.dls.transform

(#2) [[noop:
encodes: (object,object) -> noopdecodes: , PILBase.create:
encodes: (Path,object) -> create
(str,object) -> create
(Tensor,object) -> create
(ndarray,object) -> create
(bytes,object) -> create
(Image,object) -> createdecodes: ],parent_label:
encodes: (object,object) -> parent_labeldecodes: ]

In [5]:
# extract labels from learner
labels = learn.dls.vocab
labels

['ABBOTTS BABBLER', 'ABBOTTS BOOBY', 'ABYSSINIAN GROUND HORNBILL', 'AFRICAN CROWNED CRANE', 'AFRICAN EMERALD CUCKOO', 'AFRICAN FIREFINCH', 'AFRICAN OYSTER CATCHER', 'AFRICAN PIED HORNBILL', 'AFRICAN PYGMY GOOSE', 'ALBATROSS', 'ALBERTS TOWHEE', 'ALEXANDRINE PARAKEET', 'ALPINE CHOUGH', 'ALTAMIRA YELLOWTHROAT', 'AMERICAN AVOCET', 'AMERICAN BITTERN', 'AMERICAN COOT', 'AMERICAN DIPPER', 'AMERICAN FLAMINGO', 'AMERICAN GOLDFINCH', 'AMERICAN KESTREL', 'AMERICAN PIPIT', 'AMERICAN REDSTART', 'AMERICAN ROBIN', 'AMERICAN WIGEON', 'AMETHYST WOODSTAR', 'ANDEAN GOOSE', 'ANDEAN LAPWING', 'ANDEAN SISKIN', 'ANHINGA', 'ANIANIAU', 'ANNAS HUMMINGBIRD', 'ANTBIRD', 'ANTILLEAN EUPHONIA', 'APAPANE', 'APOSTLEBIRD', 'ARARIPE MANAKIN', 'ASHY STORM PETREL', 'ASHY THRUSHBIRD', 'ASIAN CRESTED IBIS', 'ASIAN DOLLARD BIRD', 'ASIAN GREEN BEE EATER', 'ASIAN OPENBILL STORK', 'AUCKLAND SHAQ', 'AUSTRAL CANASTERO', 'AUSTRALASIAN FIGBIRD', 'AVADAVAT', 'AZARAS SPINETAIL', 'AZURE BREASTED PITTA', 'AZURE JAY', 'AZURE TANAGER', '

### Let's add the missing parts to our Pytorch model

As mentioned previously, FastAI learner wraps out PyTorch model. We are adding 2 additional layers to the models to achieve the same results as the FastAI learner. 

### Why can't we just export the learner to ONNX?
AFAIK FastAI currently doesn't support ONNX exports. We need to extract the PyTorch model as an intermediary step before we can convert it to ONNX. 

--------

### Additional layers:

#### Normalization layer
First we need to add a normalization layer. Pre-trained PyTorch models are trained on the ImageNet dataset for which the following normalization variables are suggested `mean=[0.485, 0.456, 0.406]`, `std=[0.229, 0.224, 0.225]`. Source: https://pytorch.org/vision/stable/models.html. Normalization is ought to increase model performance.


#### Softmax layer
By adding a softmax layer at the end of our final model definition we are making a our results 'human readble'. As en example here's an inference result without the softmax layer `('not_hot_dog', array([[-3.0275817,  1.2424631]], dtype=float32))`. *Helpful? Not really imo.*

Here's the inference with the added softmax layer: `('not_hot_dog', array([[0.01378838, 0.98621166]], dtype=float32))`.

In [6]:
import torch
import torchvision
import torchvision.transforms as transforms

# https://pytorch.org/vision/stable/models.html

pytorch_model = learn.model.eval() # gets the PyTorch model
softmax_layer = torch.nn.Softmax(dim=1) # define softmax
normalization_layer = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # normalization layer

# assembling the final model
final_model = nn.Sequential(
    normalization_layer,
    pytorch_model,
    softmax_layer
)

# Note: Image resizing will be handled separately


In [7]:
# loading an image and converting to tensor
from PIL import Image

def image_transform(path: str, size: int) -> torch.Tensor:
    '''Helper function to transform image.'''
    image = Image.open(path)

    # transformation pipeline
    transformation = transforms.Compose([
                transforms.Resize([size,size]), # resizes image
                transforms.ToTensor() # converts to image to tensor
            ])

    image_tensor = transformation(image).unsqueeze(0)
    print('Tensor shape: ', image_tensor.shape)

    return image_tensor

In [8]:
# test image paths
hot_dog_test = 'test.jpg'

In [9]:
# get image tensors
hot_dog_tensor = image_transform(hot_dog_test, 256)

Tensor shape:  torch.Size([1, 3, 256, 256])


In [10]:
# run inference on test images

with torch.no_grad():
    results = final_model(hot_dog_tensor)
labels[np.argmax(results.detach().numpy())], results.detach().numpy().astype(float)

('ROADRUNNER',
 array([[3.61593297e-13, 6.54161729e-13, 6.98207160e-12, 1.33635182e-12,
         2.54244954e-12, 2.29816817e-13, 7.94867830e-15, 5.26444166e-10,
         1.60433546e-15, 1.54276040e-14, 6.09114538e-13, 4.05555065e-13,
         3.35615799e-14, 1.46446067e-11, 1.30020430e-12, 1.57727408e-11,
         2.59590238e-14, 2.25453760e-14, 4.57062502e-14, 1.81622959e-13,
         1.83714268e-15, 3.05196528e-11, 4.65076708e-13, 2.56261375e-13,
         1.10292560e-13, 1.63227925e-14, 3.75735684e-14, 5.21765790e-14,
         1.65749554e-12, 2.46322823e-10, 6.55221193e-15, 4.52085360e-16,
         1.22464233e-10, 9.45628667e-14, 1.33251471e-14, 1.18210747e-12,
         2.43996368e-11, 4.52914150e-15, 6.30031780e-13, 1.30779448e-13,
         7.86293909e-15, 1.54331821e-12, 3.46076764e-13, 8.97090874e-11,
         1.52843316e-09, 2.28931128e-11, 1.42041337e-12, 1.66764850e-13,
         1.20787885e-11, 2.64698592e-14, 1.27930668e-16, 2.35566720e-13,
         5.73026056e-13, 2.85347273e

## 2. Export PyTorch model to ONNX

It's REALLY important to define the input shape of an ONNX model. We trained the FastAI/PyTorch model on 256 x 256 image. And we need to use the same for export. The input tensor must be in BCHW format - 1x3x256x256. (Batch x Channels x Height x Width).

PyTorch documentation: https://pytorch.org/docs/master/onnx.html

In [11]:
torch.onnx.export(
    final_model, 
    torch.randn(1, 3, 256, 256),
    "models/bird_model2_resnet34_256_256.onnx",
    do_constant_folding=True,
    export_params=True, # if set to False exports untrained model
    input_names=["image_1_3_256_256"],
    output_names=["bird"],
    opset_version=11
)

### (Optional) Validate ONNX model

In [12]:
import onnx

# Load the ONNX model
model = onnx.load('models/bird_model2_resnet34_256_256.onnx')

# Check that the IR is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
# onnx.helper.printable_graph(model.graph)

We got out ONNX model. Let's compare the results.

In [13]:
import numpy as np
import onnxruntime as rt

np.set_printoptions(suppress=True)

In [14]:
from PIL import Image

def image_transform_onnx(path: str, size: int) -> np.ndarray:
    '''Image transform helper for onnx runtime inference.'''

    image = Image.open(path)
    image = image.resize((size,size))
    # print(image.shape, image.mode)


    # now our image is represented by 3 layers - Red, Green, Blue
    # each layer has a 224 x 224 values representing
    image = np.array(image)
    # print('Conversion to tensor: ',image.shape)

    # dummy input for the model at export - torch.randn(1, 3, 224, 224)
    image = image.transpose(2,0,1).astype(np.float32)
    # print('Transposing the tensor: ',image.shape)

    # our image is currently represented by values ranging between 0-255
    # we need to convert these values to 0.0-1.0 - those are the values that are expected by our model

    # print('Integer value: ', image[0][0][40])
    image /= 255
    # print('Float value: ', image[0][0][40])

    # expanding the alread existing tensor with the final dimension (similar to unsqueeze(0))
    # currently our tensor only has rank of 3 which needs to be expanded to 4 - torch.randn(1, 3, 224, 224)
    # 1 can be considered the batch size

    image = image[None, ...]
    # print('Final shape of our tensor', image.shape, '\n')
    return image


In [15]:
hot_dog_tensor_onnx = image_transform_onnx('test.jpg', 256)

In [16]:
# initialize onnx runtime inference session
sess = rt.InferenceSession('models/bird_model2_resnet34_256_256.onnx')

# input & output names
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name

# input dimensions
input_dims = sess.get_inputs()[0].shape

input_name, output_name, input_dims

('image_1_3_256_256', 'bird', [1, 3, 256, 256])

In [23]:
results = sess.run([output_name], {input_name: hot_dog_tensor_onnx})[0]
labels[np.argmax(results)], results, labels

('COCKATOO',
 array([[0.00000002, 0.00000285, 0.00000002, 0.00000007, 0.00000034,
         0.00000001, 0.00000002, 0.00000001, 0.00000017, 0.00000268,
         0.00000002, 0.00013275, 0.00000061, 0.        , 0.00000033,
         0.        , 0.00000001, 0.00000008, 0.00000002, 0.00000007,
         0.00000013, 0.00000001, 0.00000009, 0.00000033, 0.00000013,
         0.00000001, 0.00000111, 0.00000001, 0.00000006, 0.        ,
         0.0000009 , 0.00000022, 0.        , 0.00000001, 0.00000006,
         0.        , 0.00000079, 0.00000042, 0.        , 0.00000235,
         0.        , 0.00000002, 0.00000007, 0.00000002, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.00000003,
         0.00000181, 0.0000011 , 0.        , 0.00000087, 0.        ,
         0.00044059, 0.00000004, 0.        , 0.        , 0.00000002,
         0.        , 0.00000119, 0.        , 0.00004179, 0.00000004,
         0.        , 0.00000048, 0.00000001, 0.00000001, 0.00000002,
         0.00000118, 

**If correct, all three model versions (FastAI, PyTorch, ONNX) have the same results (with minor differences).**

## Inference time

One of the advantages of ONNX runtime - in most of the cases it's faster than it's original format. Let's see if that holds up.

### FastAI

In [17]:
%time
learn.predict('test.jpg')

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.44 µs


('ROADRUNNER',
 tensor(422),
 tensor([6.5983e-14, 2.2859e-13, 5.9723e-12, 7.7789e-13, 1.5575e-12, 3.2188e-14,
         7.9489e-15, 1.3908e-10, 3.5392e-16, 8.0554e-15, 8.7294e-14, 4.9873e-14,
         1.5486e-14, 1.5034e-12, 9.4583e-14, 6.2127e-13, 6.7510e-15, 4.8466e-15,
         7.4699e-15, 5.3998e-14, 7.7535e-16, 1.0335e-11, 2.3246e-13, 1.1478e-13,
         4.0351e-14, 4.7952e-15, 1.2747e-14, 3.7469e-14, 4.8960e-13, 2.5345e-11,
         2.9240e-15, 1.8969e-16, 3.9972e-11, 3.6214e-14, 2.6427e-15, 2.2619e-13,
         6.9356e-12, 9.2867e-16, 8.5566e-13, 1.8381e-14, 5.1385e-15, 2.0151e-13,
         1.2745e-13, 5.1696e-11, 4.9146e-10, 4.1472e-11, 2.5322e-13, 1.6292e-14,
         1.3606e-11, 1.6115e-14, 9.0644e-17, 9.7111e-14, 1.5005e-13, 3.6555e-15,
         9.6599e-13, 3.9306e-13, 8.2510e-12, 2.9404e-15, 2.1990e-09, 1.0551e-13,
         3.6785e-14, 4.3111e-15, 9.5979e-11, 2.9967e-15, 7.0435e-12, 2.3685e-10,
         8.6018e-13, 4.7681e-13, 2.7982e-12, 8.5201e-12, 1.5925e-15, 1.1203e-12,

### PyTorch

In [18]:
%time

hot_dog_tensor = image_transform('test.jpg', 256)

with torch.no_grad():
    results = final_model(hot_dog_tensor)
labels[np.argmax(results.detach().numpy())], results.detach().numpy().astype(float)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs
Tensor shape:  torch.Size([1, 3, 256, 256])


('ROADRUNNER',
 array([[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        

### ONNX

In [19]:
%time

hot_dog_tensor_onnx = image_transform_onnx('test.jpg', 256)
results = sess.run([output_name], {input_name: hot_dog_tensor_onnx})[0]
labels[np.argmax(results)], results, labels

CPU times: user 15 µs, sys: 0 ns, total: 15 µs
Wall time: 26.2 µs


('ROADRUNNER',
 array([[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.00000001, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        

No real difference between the three. 

## Gotchas

It's really important to use the expected input with ONNX. Let's check the following scenario when using a 1x3x224x224 tensor with on a model with a defined input of 1x3x256x256. I've been struggling to figure out the correct ONNX settings. I hope this will help to some of you.

Let's see what happens

In [20]:
hot_dog_tensor_onnx = image_transform_onnx('test.jpg', 224)

In [21]:
# This will throw an error becuase of incorrect input size.

results = sess.run([output_name], {input_name: hot_dog_tensor_onnx})[0]
labels[np.argmax(results)], results, labels

InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Got invalid dimensions for input: image_1_3_256_256 for the following indices
 index: 2 Got: 224 Expected: 256
 index: 3 Got: 224 Expected: 256
 Please fix either the inputs or the model.

## How to Debug

To check the inputs of a model you can use a tool like Netron to visualize it: https://netron.app. Desktop version available here: https://github.com/lutzroeder/netron

OR you can access the expected dimension by the following line:


In [22]:
# shows the required model input
sess.get_inputs()[0].shape

[1, 3, 256, 256]

## You got your own ONNX model.